<a href="https://colab.research.google.com/github/Hemachandran-D45/Python/blob/main/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-learn -q
!pip install numpy -q
!pip install pandas -q
!pip install matplotlib -q
!pip install seaborn -q
!pip install scipy -q
!pip install missingo -q
!pip install fasteda -q

ERROR: Could not find a version that satisfies the requirement missingo (from versions: none)
ERROR: No matching distribution found for missingo
  Preparing metadata (setup.py) ... done


In [3]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from scipy.stats import shapiro

#Regression

Predict the diabete score after thir bp,bmi,sugar level recorded.

#Loading a data
The following code loads the diabetes data set from **sklearn** and prints out a description of the data.

In [4]:
from sklearn.datasets import load_diabetes, load_iris

In [5]:
# Load the diabetes dataset with return_X_y=True and as_frame=True to get pandas DataFrames
diabetes_x,diabetes_y = load_diabetes(return_X_y=True, as_frame=True, scaled=False)

#Concatenate the features (diabetes_X) and target (diabetes_y) into one DataFrame
diabetes = pd.concat([diabetes_x, pd.Series(diabetes_y)], axis=1).rename({0: 'target'},axis=1)

# Load the diabetes dataset again to print the description (without return_X_y)
diabetes_default = load_diabetes()

# Print the data set description:
print(diabetes_default['DESCR'])

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

:Number of Instances: 442

:Number of Attributes: First 10 columns are numeric predictive values

:Target: Column 11 is a quantitative measure of disease progression one year after baseline

:Attribute Information:
    - age     age in years
    - sex
    - bmi     body mass index
    - bp      average blood pressure
    - s1      tc, total serum cholesterol
    - s2      ldl, low-density lipoproteins
    - s3      hdl, high-density lipoproteins
    - s4      tch, total cholesterol / HDL
    - s5      ltg, possibly log of serum triglycerides level
    - s6      glu, blood sugar level

Note: Each of these 10 feature variables have bee

In [6]:
#Randomly take 5 row as sample
diabetes.sample(5)

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
325,48.0,1.0,31.4,75.33,242.0,151.6,38.0,6.37,5.5683,103.0,192.0
189,48.0,1.0,20.2,95.00,187.0,117.4,53.0,4.00,4.4188,85.0,79.0
312,28.0,1.0,24.2,93.00,174.0,106.4,54.0,3.00,4.2195,84.0,144.0
112,25.0,1.0,28.3,87.00,193.0,128.0,49.0,4.00,4.3820,92.0,246.0
281,23.0,2.0,18.0,78.00,171.0,96.0,48.0,4.00,4.9053,92.0,94.0


In [7]:
#Check whether there is a any missing data
diabetes.isna().max(axis=0).max()
#if there atleast one missing also it give TRUE

False

In [8]:
diabetes.isna().any().any()

False

In [9]:
#Sets a seed for the random number generator to ensure that the random selections are reproducible.
random.seed(2024)
#select 3 ramdon column
missing_cols = random.sample(range(len(diabetes.columns) - 1), 3)

#Selecting 10% of Rows Randomly:
missing_rows = random.sample(diabetes.index.tolist(),int(np.round(len(diabetes.index.tolist())/10)))

#selecting specific cell to missing values
diabetes.iloc[missing_rows,missing_cols] = np.nan

In [11]:
print(sorted(diabetes.columns[missing_cols]))

['bmi', 's1', 's4']


**Preprocessing**

Normally preprocessing conducted after EDA but here we going to do it now.

**One-hot Encoding**

convert categorial data to binary format [0,1]

In [13]:
from sklearn.preprocessing import OneHotEncoder
#handle unknown ignore is to avoid error and drop none is to ensure all categories are retain
enc1 = OneHotEncoder(handle_unknown='ignore', drop=None)

# #change the sex column from df to numpy array
encoded_sex = enc1.fit_transform(diabetes[['sex']]).toarray()

#again convert that to df with new column based on categories in sex (sex1,sex2)
encoded_sex = pd.DataFrame(encoded_sex, columns=['sex' + str(int(x)) for x in enc1.categories_[0]])
#now append that column
diabetes = pd.concat([diabetes, encoded_sex], axis=1)

# Sample 10 rows. Print only the 'sex', 'sex1', and 'sex2' columns for simplicity
diabetes[['sex', 'sex1', 'sex2']].sample(10)

,sex,sex1,sex2
373,1.0,1.0,0.0
335,1.0,1.0,0.0
277,1.0,1.0,0.0
257,2.0,0.0,1.0
340,1.0,1.0,0.0
98,2.0,0.0,1.0
229,2.0,0.0,1.0
383,2.0,0.0,1.0
222,2.0,0.0,1.0
276,1.0,1.0,0.0
